In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense

import pandas as pd
import traceback
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
os.environ['KMP_DUPLICATE_LIB_OK']='True'


Using TensorFlow backend.


In [0]:
def read_UTK(startpath):
    X = []
    y = []
    try:
        for root, dirs, files in os.walk(startpath):
            for file in files:
                if file[-3:] == 'jpg':
                    names = file.split('_')
                    if int(names[0]) > 17 and int(names[0]) < 35:
                      
                      y.append(names[1])
                      img = cv2.imread(root + file,cv2.IMREAD_GRAYSCALE)
                      X.append(img)
                
    except Exception as err:
        traceback.print_tb(err.__traceback__)
    return np.array(X,dtype='float'), np.array(y,dtype='uint8')

In [0]:
UTK_path = 'UTKFace/'
X,y = read_UTK(UTK_path)
y_translation = {
    0: 'Male',
    1: 'Female',
}

In [0]:
X = np.expand_dims(X, -1)
oht = OneHotEncoder(categories='auto', sparse=False)
y = oht.fit_transform(y.reshape(-1,1))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
classes = 2
inputShape = (200,200,1)

In [0]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=inputShape, activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(classes, activation = 'softmax'))

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
model.fit(X_train, y_train, batch_size=200, epochs=12)

Epoch 1/12
7495/7495 [==============================] - 15s 2ms/step - loss: 0.1286 - acc: 0.9581
Epoch 2/12
7495/7495 [==============================] - 15s 2ms/step - loss: 0.1131 - acc: 0.9646
Epoch 3/12
7495/7495 [==============================] - 15s 2ms/step - loss: 0.1191 - acc: 0.9626
Epoch 4/12
7495/7495 [==============================] - 15s 2ms/step - loss: 0.1177 - acc: 0.9616
Epoch 5/12
7495/7495 [==============================] - 15s 2ms/step - loss: 0.0965 - acc: 0.9704
Epoch 6/12
7495/7495 [==============================] - 15s 2ms/step - loss: 0.0810 - acc: 0.9744
Epoch 7/12
7495/7495 [==============================] - 15s 2ms/step - loss: 0.0819 - acc: 0.9709
Epoch 8/12
7495/7495 [==============================] - 15s 2ms/step - loss: 0.0764 - acc: 0.9761
Epoch 9/12
7495/7495 [==============================] - 15s 2ms/step - loss: 0.0642 - acc: 0.9811
Epoch 10/12
7495/7495 [==============================] - 15s 2ms/step - loss: 0.0636 - acc: 0.9791
Epoch 11/12
7495/74

In [0]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Testing loss: {}, acc: {}\n'.format(loss, acc))

Testing loss: 0.19390694362719973, acc: 0.9391756702681072



In [0]:
model.save('gender_classifier_model')

In [0]:
from joblib import dump, load
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.decomposition import PCA

In [0]:
svc = SVC()
pca = PCA(100)
X_train_reduced = pca.fit_transform(X_train.reshape(X_train.shape[0], -1))
X_test_reduced = pca.transform(X_test.reshape(X_test.shape[0], -1))

In [0]:
logreg = LogisticRegression()
nb = BernoulliNB()

In [11]:
svc.fit(X_train_reduced, np.argmax(y_train, axis=1))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [12]:
svc.score(X_test_reduced, np.argmax(y_test, axis=1))

0.5762304921968787

In [13]:
logreg.fit(X_train_reduced, np.argmax(y_train, axis=1))
logreg.score(X_test_reduced, np.argmax(y_test, axis=1))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.902360944377751

In [14]:
nb.fit(X_train_reduced, np.argmax(y_train, axis=1))
nb.score(X_test_reduced, np.argmax(y_test, axis=1))

0.8011204481792717